<a href="https://colab.research.google.com/github/RayBasilio123/IC_2022/blob/main/hyperparametros_Grid_Arvore_Maceio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
import pandas as pd
import math
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import f1_score
from datetime import datetime
from matplotlib import pyplot

from math import nan
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV


In [16]:
def get_x2(df,lista,lags,Eto,lags_eto):
  lags=lags
  lista_aux=[]
  lista_aux2=[]
  max_lag=0;
  data = pd.DataFrame()
  eta_nois = pd.DataFrame()
  eta_nois['Data']=df['Data']
  for coluna in lista:
    data[coluna] = df[coluna]       

  for i in range(len(lags)):
    for j in range(len(lags[i])):
      lista_aux = data.iloc[:,i].tolist()
      for displacement in range((lags[i][j])):
        if max_lag<(lags[i][j]):
          max_lag=(lags[i][j])
        del lista_aux[len(lista_aux)-1]
        lista_aux.insert(0,nan)
      eta_nois[((data.iloc[:,i]).name)+("_t-")+str(lags[i][j])]=(lista_aux)  
  
  for i in range(len(lags_eto)):
    lista_aux2=df[Eto].iloc[:,0].tolist()
    for displacement in range((lags_eto[i])):
      if max_lag<(lags_eto[i]):
        max_lag=(lags_eto[i])
      del lista_aux2[len(lista_aux2)-1]
      lista_aux2.insert(0,nan)
    eta_nois[((df[Eto].iloc[:,0]).name)+("_t-")+str(lags_eto[i])]=lista_aux2        
  return eta_nois,max_lag,lista,lags,Eto,lags_eto;

In [17]:
def get_x30(df,lista, Eto):
  ix = []
  idx =  [i for i in np.arange(1, 31)]
  for i in range(len(lista)):
    ix.append(idx)

  resultado = get_x2(df, lista, ix, Eto, idx)
  return resultado

In [18]:
def column_Filter(df,dias):
  list_column = []
  # pegando colunas do data frame
  for column in (df.columns):
    if column != 'Data':
      list_column.append(column)
  # prosseguindo com a filtração
  b=[]
  lags=[]
  aux3=[]
  lista=[]
  Eto=[]
  lags_eto=[]
 
  # filtra e  remove colunas 
  for i in range(len(list_column)):
    j=list_column[i].split("_t-")
    if (int(j[1])>=dias):
      b.append(j)
  # print("Lista_Filtrada",b)

  #localiza as colunas  e faz uma lista se a coluna for Eto cria uma lista só pra ela
  for k in range(len(b)):
    if b[k][0] not in lista and b[k][0]!="Eto":
        lista.append(b[k][0])
    elif b[k][0] not in Eto and b[k][0]=="Eto":
        Eto.append(b[k][0])

 
  # localiza as colunas e cria uma lista de lags para cada
  for i in range(len(lista)):
    aux=[]
    for k in range(len(b)):
      if (lista[i]==b[k][0])  :
          aux.append(int(b[k][1]))
    lags.append(aux)
    del(aux)
  
  
  if Eto!=[]:
    for k in range(len(b)):
      if b[k][0]==Eto[0]:
        lags_eto.append(int(b[k][1]))
 
  aux3.append(lista)
  aux3.append(lags)
  aux3.append(Eto)
  aux3.append(lags_eto)
 

  return aux3









In [19]:
def resource_ranking(df,lista_filtrada3,variavel_alvo) :
  
  # resource_ranking(df_la,lista_filtrada2,"temp_min")  
  #Monta a tabela com os dados filtrados 
  
  tab=get_x2(df,lista_filtrada3[0],lista_filtrada3[1],lista_filtrada3[2],lista_filtrada3[3])
 
  # dff=tab[0].drop("Data",axis=1)
  
  # train_size = int(((len(tab[0]))-tab[1]) )
  # dff=dff.iloc[tab[1]:train_size,:]

  # array1 = dff.values

  # df= df.drop("Data",axis=1)
  # df=df.iloc[tab[1]:train_size,:]
  # array2 = df[variavel_alvo]
  selecao_treino = (tab[0]['Data'] >= '2014') & (tab[0]['Data'] <= '2018-12-31')
  
  dff=tab[0][selecao_treino].drop("Data",axis=1)

  train_size = int(((len(tab[0][selecao_treino]))-tab[1]) )
  dff=dff.iloc[tab[1]:train_size,:]
  array1 = dff.values

  df= df[selecao_treino].drop("Data",axis=1)
  df=df.iloc[tab[1]:train_size,:]
  array2 = df[variavel_alvo]

 

  X =array1[:,0:len(tab[0].columns)]
  Y = array2
  # feature extraction
  test = SelectKBest(score_func=f_regression, k=4)
  fit = test.fit(X, Y)
  # summarize scores
  set_printoptions(precision=3)
 

    
  # print("Selecao_univariada",fit.scores_)
  f=fit.scores_
  f_ord = sorted(f,reverse=True)
  # print("Selecao_univariada_ordenada",f_ord)

  ll=[]
  for y in range(len(f_ord)):
    for i in range(len(f)):
      if (f_ord[y]==f[i]):
        ll.append(i)
        # print("f[i]",f[i])
        # print("f_ord[y]",f_ord[y])
  leg_seq=[]
  for i in range(len(ll)):
    leg_seq.append(dff.columns[ll[i]])    

  # print("Colunas_selecionadas [0]- Selecao_univariada",leg_seq)
 
  model = ExtraTreesRegressor(n_estimators=10,random_state=42)
  model.fit(X,Y)
  g=model.feature_importances_
  g_ord=sorted(model.feature_importances_,reverse=True)
  # print("Importancia",g)
  # print("Importancia_ordenada",g_ord)
  jj=[]
  for y in range(len(g_ord)):
    for i in range(len(g)):
      if (g_ord[y]==g[i]):
        jj.append(i)
  leg_seq2=[]
  for i in range(len(ll)):
    leg_seq2.append(dff.columns[jj[i]])    
  # print("Colunas_selecionadas [1]- Importância do recurso",leg_seq2)
  return leg_seq,leg_seq2

In [20]:


def list_Format(melhores_recursos,q,dias):
  b=[]
  lags=[]
  aux3=[]
  lista=[]
  Eto=[]
  lags_eto=[]
  q = len(melhores_recursos) if q==0 else q
  # filtra e  remove colunas 
  for i in range(q):
    j=melhores_recursos[i].split("_t-")
    if (int(j[1])>=dias):
      b.append(j)
  print("lista_selecionada",b)

  #localiza as colunas  e faz uma lista se a coluna for Eto cria uma lista só pra ela
  for k in range(len(b)):
    if b[k][0] not in lista and b[k][0]!="Eto":
        lista.append(b[k][0])
    elif b[k][0] not in Eto and b[k][0]=="Eto":
        Eto.append(b[k][0])

  print("lista",lista)
  print("Eto",Eto)
 
  # localiza as colunas e cria uma lista de lags para cada
  for i in range(len(lista)):
    aux=[]
    for k in range(len(b)):
      if (lista[i]==b[k][0])  :
          aux.append(int(b[k][1]))
    lags.append(aux)
    del(aux)

  print("lags",lags)
           
  # print("Eto",Eto)
  
  
  if Eto!=[]:
    for k in range(len(b)):
      if b[k][0]==Eto[0]:
        lags_eto.append(int(b[k][1]))
  print("lags_eto",lags_eto)
 
  aux3.append(lista)
  aux3.append(lags)
  aux3.append(Eto)
  aux3.append(lags_eto)
 

  return aux3

In [21]:
def train_test(df, lista,lags,Eto,lags_eto,variavel_Alvo,data_Itest= '2019-01-01',data_Ftest= '2020-12-31',data_Itreino='2014-01-01',data_Ftreino='2018-12-31'):
  tabela = get_x2(df, lista,lags,Eto,lags_eto)
  selecao_treino = (tabela[0]['Data'] >= data_Itreino) & (tabela[0]['Data'] <= data_Ftreino)
  selecao_teste = (tabela[0]['Data'] >= data_Itest) & (tabela[0]['Data'] <= data_Ftest)

  x1_train = tabela[0][selecao_treino].drop("Data", axis=1)
  x1_test = tabela[0][selecao_teste].drop("Data", axis=1)
  
  y1_train = df[variavel_Alvo][selecao_treino]
  y1_test = df[variavel_Alvo][selecao_teste]

  x1_train = x1_train[tabela[1]:]
  y1_train = y1_train[tabela[1]:]

  
  print("***************************************")
  print(x1_train.shape,"x1_train")
  #[6569 rows x 10 columns] x1_train
  print("*****************++*********************")
  print(x1_test.shape,"x1_test")
  print("******************---********************")
  print(y1_train.shape,"y1_train")
  print("****************1234*********************")
  print(y1_test.shape,"y1_test")
  print("*****************asddfg******************")
 
  
  return x1_train, x1_test,y1_train, y1_test

In [22]:
def arvore(df,lista, lags, Eto, lags_eto,variavel_Alvo):
  
  x1_train, x1_test,y1_train, y1_test = train_test(df, lista,lags,Eto,lags_eto,variavel_Alvo)
  # print("***************************************")
  # print(x1_train,"x1_train")
  # #[6569 rows x 10 columns] x1_train
  # print("*****************++*********************")
  # print(x1_test,"x1_test")
  # print("******************---********************")
  # print(y1_train,"y1_train")
  # print("****************1234*********************")
  # print(y1_test,"y1_test")
  # print("*****************asddfg******************")

  model = DecisionTreeRegressor(random_state=42)
  model.fit(x1_train, y1_train)
  print("------------------")
  print("Arvore")
  y1_pred = model.predict(x1_test)
  mse = mean_squared_error(y1_test, y1_pred)
  std_mse=np.std(np.sqrt((y1_pred - y1_test)**2))
  
  print("std_mse",round(std_mse,4))
  rmse = math.sqrt(mse)
  print("Erro medio absoluto----",round(mean_absolute_error(y1_test, y1_pred),4))
  # pyplot.plot(y1_test, label='Expected tree ')
  pyplot.plot(np.arange(y1_test.shape[0]),y1_test, label='Expected tree ')
  # pyplot.plot(df["Eto"], label='variavel_Alvo ')
  pyplot.plot(y1_pred, label='Predicted tree')
  pyplot.legend(bbox_to_anchor=(0,1.02,1,0.2), loc="lower left",
                mode="expand", borderaxespad=0, ncol=3)
  pyplot.show()
  return lista,lags,Eto,lags_eto,round(rmse,4)

def arvores(df,arvore_parametros,variavel_Alvo):
  lista_colunas=["lista","lista_lags","Eto","lags_eto","rmse"]
  tb = pd.DataFrame(columns=lista_colunas)
  print("Arvores")
  for x in range(len(arvore_parametros)):
    a=arvore(df,arvore_parametros[x][0],arvore_parametros[x][1],arvore_parametros[x][2],arvore_parametros[x][3],variavel_Alvo)
 
    tb.loc[x,'lista']=a[0]
    tb.loc[x,'lista_lags']=a[1]
    tb.loc[x,'Eto']=a[2]
    tb.loc[x,'lags_eto']=a[3]
    tb.loc[x,"rmse"]=a[4]

  print(tb)

  return tb

In [23]:
def gridSearchF(df, lista,lags,Eto,lags_eto,variavel_Alvo):
  x1_train, x1_test,y1_train, y1_test = train_test(df, lista,lags,Eto,lags_eto,variavel_Alvo)
   # Create the parameter grid based on the results of random search 
  param_grid = {
      'bootstrap': [True],
      'max_depth': [10,80, 90, 100, 110],
      'max_features': [2, 3],
      'min_samples_leaf': [3, 4, 5],
      'min_samples_split': [8, 10, 12],
      'n_estimators': [100, 200, 300, 1000]
  }

  # Create a base model
  rf = RandomForestRegressor(random_state = 42)

  # Instantiate the grid search model
  grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                            cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)

  print(" -----------------grid_search.fit--------------")
 
  start_time=timer(None)

  grid_search.fit(x1_train, y1_train);

  timer(start_time)  
  
    
  return rf_random.best_params_

def gridSearchFs(df,parametros,variavel_Alvo):

  lista_colunas=["gridSearch"]
  tb = pd.DataFrame(columns=lista_colunas)
  
  print("gridSearchFs")
  for x in range(len(parametros)):
    a=RandomizedSearchF(df,parametros[x][0],parametros[x][1],parametros[x][2],parametros[x][3],variavel_Alvo)
 
    tb.loc[x,'Best_ramdon_hyperparameter']=[a]
  print('---------------------------------------------------------------------------------')
  

  return tb

# gridSearch Arvore

def gridSearchA(df, lista,lags,Eto,lags_eto,variavel_Alvo):
  tb = pd.DataFrame()
  x1_train, x1_test,y1_train, y1_test = train_test(df, lista,lags,Eto,lags_eto,variavel_Alvo)
   # Create the parameter grid based on the results of random search 
  
  param_grid={"splitter":["best","random"],
            "max_depth" : [1,3,5,7,9,11,12],
            "min_samples_leaf":[1,2,3,4,5,6,7,8,9,10],
            "min_weight_fraction_leaf":[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
            "max_features":["auto","log2","sqrt",None],
            "max_leaf_nodes":[None,10,20,30,40,50,60,70,80,90] 
            }
 
  # Create a base model
  rf = DecisionTreeRegressor(random_state = 42)

  # Instantiate the grid search model
  grid_search=GridSearchCV(rf,param_grid=param_grid,scoring='neg_mean_squared_error',cv=3,verbose=3,n_jobs = -1,return_train_score=True)
  

  print(" -----------------grid_search.fit--------------")
  start_time=timer(None)

  grid_search.fit(x1_train, y1_train);

  timer(start_time)  

  return grid_search.best_params_

def gridSearchAs(df,parametros,variavel_Alvo):
  print('---------------------------------------------------------------------------------')
  
  for x in range(len(parametros)):
    a=gridSearchA(df,parametros[x][0],parametros[x][1],parametros[x][2],parametros[x][3],variavel_Alvo)
    
    print(x,"Best_hyperparameter_Arvore", a)
   
  print('---------------------------------------------------------------------------------')
  

  return a


In [24]:
def timer(start_time=None):
    if not start_time:
        start_time=datetime.now()
        return start_time
    elif start_time:
        thour,temp_sec=divmod((datetime.now()-start_time).total_seconds(),3600)
        tmin,tsec=divmod(temp_sec,60)

In [25]:
url=('https://raw.githubusercontent.com/RayBasilio123/database/main/INMET/CSV/J_Database/Maceio_J_2014_2020.csv')
df = pd.read_csv(url, sep=",", encoding = "ISO-8859-1")
df=df.drop(columns=['Unnamed: 0'])

In [26]:
df

,Data,vento,radiacao,temp_max,temp_min,umi_max,umi_min,umi_rel,press_atm,temp_media,Eto,J
0,2014-01-01,2.306182,1.747758,31.2,22.2,92.0,46.0,73.458333,1003.866667,26.287500,4.159927,1
1,2014-01-02,2.253203,1.122256,31.3,21.8,91.0,51.0,74.041667,1002.812500,25.887500,4.020828,2
2,2014-01-03,2.088030,1.112062,31.5,20.8,91.0,48.0,70.666667,1003.445833,26.027083,4.206571,3
3,2014-01-04,1.960255,1.230881,31.4,20.9,90.0,42.0,69.958333,1005.116667,25.922917,4.242885,4
4,2014-01-05,1.872994,1.083530,31.3,20.8,90.0,47.0,71.875000,1005.154167,25.808333,3.857614,5
...,...,...,...,...,...,...,...,...,...,...,...,...
2552,2020-12-27,2.910776,0.693007,30.0,22.6,100.0,60.0,82.541667,1004.208333,25.912500,3.117960,362
2553,2020-12-28,2.832865,1.005733,30.3,24.4,98.0,49.0,76.000000,1004.708333,26.702083,4.164469,363
2554,2020-12-29,1.601862,0.984817,30.7,21.4,100.0,50.0,78.208333,1003.200000,25.637500,3.208941,364
2555,2020-12-30,1.611211,0.890091,30.9,21.8,100.0,55.0,80.250000,1002.975000,25.952083,2.808950,365


In [27]:
atributeP= [ 'vento'	,'radiacao',	'temp_max',	'temp_min',	'umi_max',	'umi_min',	'umi_rel',	'press_atm',	'temp_media','J']
resultadoP=get_x30(df,atributeP,["Eto"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [28]:
for x in [1,3,7,10]:
 lista_Filtrada=column_Filter(resultadoP[0],x)
 ranking_data = resource_ranking(df,lista_Filtrada,'Eto')
 print(ranking_data)
 lista_Formatada1=list_Format(ranking_data[0],10,0)
 lista_Formatada2=list_Format(ranking_data[1],10,0)
 print('\n -----------Para----' + str(x) + '----dias --------\n')
 print("lista_Formatada_1 -------> ",lista_Formatada1)
 print("lista_Formatada_2 -------> ",lista_Formatada2)
 print("lista_Formatada_3 -------> ",lista_Filtrada)
 print('\n ---------------------------------------------\n')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


(['Eto_t-1', 'Eto_t-2', 'umi_rel_t-1', 'Eto_t-3', 'radiacao_t-1', 'vento_t-1', 'Eto_t-4', 'temp_max_t-1', 'temp_media_t-1', 'Eto_t-5', 'umi_min_t-1', 'Eto_t-6', 'Eto_t-7', 'Eto_t-9', 'radiacao_t-2', 'Eto_t-10', 'Eto_t-8', 'Eto_t-11', 'Eto_t-12', 'umi_rel_t-2', 'Eto_t-13', 'Eto_t-14', 'radiacao_t-3', 'temp_max_t-2', 'Eto_t-15', 'Eto_t-21', 'Eto_t-23', 'Eto_t-19', 'vento_t-2', 'Eto_t-22', 'temp_media_t-2', 'Eto_t-20', 'Eto_t-16', 'Eto_t-24', 'umi_rel_t-3', 'radiacao_t-4', 'Eto_t-18', 'Eto_t-17', 'umi_min_t-2', 'Eto_t-25', 'vento_t-3', 'temp_max_t-3', 'umi_rel_t-4', 'vento_t-4', 'temp_media_t-3', 'radiacao_t-7', 'radiacao_t-6', 'Eto_t-26', 'radiacao_t-5', 'radiacao_t-8', 'radiacao_t-12', 'umi_rel_t-5', 'Eto_t-27', 'vento_t-5', 'umi_rel_t-9', 'vento_t-23', 'temp_max_t-4', 'Eto_t-28', 'umi_rel_t-6', 'temp_media_t-4', 'radiacao_t-9', 'radiacao_t-13', 'vento_t-21', 'umi_rel_t-7', 'vento_t-22', 'umi_rel_t-8', 'umi_rel_t-10', 'vento_t-24', 'umi_min_t-3', 'vento_t-6', 'Eto_t-29', 'radiacao_t-11'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


(['Eto_t-3', 'Eto_t-4', 'Eto_t-5', 'Eto_t-6', 'Eto_t-7', 'Eto_t-9', 'Eto_t-10', 'Eto_t-8', 'Eto_t-11', 'Eto_t-12', 'Eto_t-13', 'Eto_t-14', 'radiacao_t-3', 'Eto_t-15', 'Eto_t-21', 'Eto_t-23', 'Eto_t-19', 'Eto_t-22', 'Eto_t-20', 'Eto_t-16', 'Eto_t-24', 'umi_rel_t-3', 'radiacao_t-4', 'Eto_t-18', 'Eto_t-17', 'Eto_t-25', 'vento_t-3', 'temp_max_t-3', 'umi_rel_t-4', 'vento_t-4', 'temp_media_t-3', 'radiacao_t-7', 'radiacao_t-6', 'Eto_t-26', 'radiacao_t-5', 'radiacao_t-8', 'radiacao_t-12', 'umi_rel_t-5', 'Eto_t-27', 'vento_t-5', 'umi_rel_t-9', 'vento_t-23', 'temp_max_t-4', 'Eto_t-28', 'umi_rel_t-6', 'temp_media_t-4', 'radiacao_t-9', 'radiacao_t-13', 'vento_t-21', 'umi_rel_t-7', 'vento_t-22', 'umi_rel_t-8', 'umi_rel_t-10', 'vento_t-24', 'umi_min_t-3', 'vento_t-6', 'Eto_t-29', 'radiacao_t-11', 'vento_t-10', 'radiacao_t-14', 'radiacao_t-15', 'vento_t-20', 'umi_rel_t-19', 'umi_rel_t-13', 'radiacao_t-10', 'vento_t-7', 'umi_rel_t-11', 'vento_t-25', 'umi_rel_t-12', 'umi_rel_t-23', 'temp_media_t-5', 'v

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


(['Eto_t-7', 'Eto_t-9', 'Eto_t-10', 'Eto_t-8', 'Eto_t-11', 'Eto_t-12', 'Eto_t-13', 'Eto_t-14', 'Eto_t-15', 'Eto_t-21', 'Eto_t-23', 'Eto_t-19', 'Eto_t-22', 'Eto_t-20', 'Eto_t-16', 'Eto_t-24', 'Eto_t-18', 'Eto_t-17', 'Eto_t-25', 'radiacao_t-7', 'Eto_t-26', 'radiacao_t-8', 'radiacao_t-12', 'Eto_t-27', 'umi_rel_t-9', 'vento_t-23', 'Eto_t-28', 'radiacao_t-9', 'radiacao_t-13', 'vento_t-21', 'umi_rel_t-7', 'vento_t-22', 'umi_rel_t-8', 'umi_rel_t-10', 'vento_t-24', 'Eto_t-29', 'radiacao_t-11', 'vento_t-10', 'radiacao_t-14', 'radiacao_t-15', 'vento_t-20', 'umi_rel_t-19', 'umi_rel_t-13', 'radiacao_t-10', 'vento_t-7', 'umi_rel_t-11', 'vento_t-25', 'umi_rel_t-12', 'umi_rel_t-23', 'vento_t-11', 'vento_t-9', 'radiacao_t-16', 'umi_rel_t-22', 'vento_t-15', 'umi_rel_t-21', 'vento_t-12', 'vento_t-26', 'umi_rel_t-14', 'umi_rel_t-20', 'vento_t-16', 'Eto_t-30', 'vento_t-18', 'vento_t-8', 'vento_t-19', 'radiacao_t-23', 'umi_rel_t-24', 'radiacao_t-22', 'vento_t-17', 'vento_t-27', 'radiacao_t-21', 'vento_t-28

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


(['Eto_t-10', 'Eto_t-11', 'Eto_t-12', 'Eto_t-13', 'Eto_t-14', 'Eto_t-15', 'Eto_t-21', 'Eto_t-23', 'Eto_t-19', 'Eto_t-22', 'Eto_t-20', 'Eto_t-16', 'Eto_t-24', 'Eto_t-18', 'Eto_t-17', 'Eto_t-25', 'Eto_t-26', 'radiacao_t-12', 'Eto_t-27', 'vento_t-23', 'Eto_t-28', 'radiacao_t-13', 'vento_t-21', 'vento_t-22', 'umi_rel_t-10', 'vento_t-24', 'Eto_t-29', 'radiacao_t-11', 'vento_t-10', 'radiacao_t-14', 'radiacao_t-15', 'vento_t-20', 'umi_rel_t-19', 'umi_rel_t-13', 'radiacao_t-10', 'umi_rel_t-11', 'vento_t-25', 'umi_rel_t-12', 'umi_rel_t-23', 'vento_t-11', 'radiacao_t-16', 'umi_rel_t-22', 'vento_t-15', 'umi_rel_t-21', 'vento_t-12', 'vento_t-26', 'umi_rel_t-14', 'umi_rel_t-20', 'vento_t-16', 'Eto_t-30', 'vento_t-18', 'vento_t-19', 'radiacao_t-23', 'umi_rel_t-24', 'radiacao_t-22', 'vento_t-17', 'vento_t-27', 'radiacao_t-21', 'vento_t-28', 'umi_rel_t-18', 'radiacao_t-24', 'umi_rel_t-15', 'radiacao_t-19', 'radiacao_t-25', 'vento_t-14', 'vento_t-29', 'radiacao_t-17', 'umi_rel_t-25', 'vento_t-13', 'umi

# Para  1 dia - Grid Arvore

In [29]:
lista_Formatada_1=  [['umi_rel', 'radiacao', 'vento', 'temp_max', 'temp_media'], [[1], [1], [1], [1], [1]], ['Eto'], [1, 2, 3, 4, 5]]
lista_Formatada_2=  [['radiacao', 'umi_min', 'umi_rel', 'J', 'temp_max'], [[1], [1], [1], [28], [1]], ['Eto'], [1, 2, 4, 5, 3]]

In [30]:
parametrosSearchA=gridSearchAs(df,[lista_Formatada_1,lista_Formatada_2],"Eto")

---------------------------------------------------------------------------------
***************************************
(1821, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1821,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 50400 candidates, totalling 151200 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
67200 fits failed out of a total of 151200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
67200 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 1320, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 304, in fit
    raise ValueError("min_weight_fraction_leaf must in [0, 0.5]")
ValueError: min_weight_fract

0 Best_hyperparameter_Arvore {'max_depth': 3, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}
***************************************
(1798, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1798,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 50400 candidates, totalling 151200 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
67200 fits failed out of a total of 151200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
67200 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 1320, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 304, in fit
    raise ValueError("min_weight_fraction_leaf must in [0, 0.5]")
ValueError: min_weight_fract

1 Best_hyperparameter_Arvore {'max_depth': 3, 'max_features': 'auto', 'max_leaf_nodes': 10, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}
---------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [-0.619 -0.73  -0.619 ...    nan    nan    nan]
  category=UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the train scores are non-finite: [-0.571 -0.663 -0.571 ...    nan    nan    nan]
  category=UserWarning,


#Para  3 dias - Grid Arvore

In [32]:
lista_Formatada_1=  [[], [], ['Eto'], [3, 4, 5, 6, 7, 9, 10, 8, 11, 12]]
lista_Formatada_2=  [['J', 'radiacao'], [[28, 25, 27, 19], [4]], ['Eto'], [3, 6, 11, 4, 8]]

In [33]:
parametrosSearchA=gridSearchAs(df,[lista_Formatada_1,lista_Formatada_2],"Eto")


---------------------------------------------------------------------------------
***************************************
(1814, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1814,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 50400 candidates, totalling 151200 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
67200 fits failed out of a total of 151200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
67200 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 1320, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 304, in fit
    raise ValueError("min_weight_fraction_leaf must in [0, 0.5]")
ValueError: min_weight_fract

0 Best_hyperparameter_Arvore {'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}
***************************************
(1798, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1798,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 50400 candidates, totalling 151200 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
67200 fits failed out of a total of 151200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
67200 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 1320, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 304, in fit
    raise ValueError("min_weight_fraction_leaf must in [0, 0.5]")
ValueError: min_weight_fract

1 Best_hyperparameter_Arvore {'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}
---------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [-0.764 -0.829 -0.764 ...    nan    nan    nan]
  category=UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the train scores are non-finite: [-0.71 -0.79 -0.71 ...   nan   nan   nan]
  category=UserWarning,


# Para 7 dias Grid Arvore

In [35]:
lista_Formatada_1=  [[], [], ['Eto'], [7, 9, 10, 8, 11, 12, 13, 14, 15, 21]]
lista_Formatada_2=  [['J'], [[27, 21, 19, 9]], ['Eto'], [13, 7, 11, 12, 8, 14]]

In [36]:
parametrosSearchA=gridSearchAs(df,[lista_Formatada_1,lista_Formatada_2],"Eto")


---------------------------------------------------------------------------------
***************************************
(1805, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1805,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 50400 candidates, totalling 151200 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
67200 fits failed out of a total of 151200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
67200 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 1320, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 304, in fit
    raise ValueError("min_weight_fraction_leaf must in [0, 0.5]")
ValueError: min_weight_fract

0 Best_hyperparameter_Arvore {'max_depth': 5, 'max_features': 'log2', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}
***************************************
(1799, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1799,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 50400 candidates, totalling 151200 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
67200 fits failed out of a total of 151200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
67200 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 1320, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 304, in fit
    raise ValueError("min_weight_fraction_leaf must in [0, 0.5]")
ValueError: min_weight_fract

1 Best_hyperparameter_Arvore {'max_depth': 5, 'max_features': 'log2', 'max_leaf_nodes': 10, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}
---------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [-0.863 -0.872 -0.863 ...    nan    nan    nan]
  category=UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the train scores are non-finite: [-0.76  -0.803 -0.76  ...    nan    nan    nan]
  category=UserWarning,


# Para 10 dias - Grid Arvore

In [38]:
lista_Formatada_1=  [[], [], ['Eto'], [10, 11, 12, 13, 14, 15, 21, 23, 19, 22]]
lista_Formatada_2=  [['J'], [[27, 24, 30, 26, 28, 29, 14]], ['Eto'], [11, 13, 15]]

In [39]:
parametrosSearchA=gridSearchAs(df,[lista_Formatada_1,lista_Formatada_2],"Eto")


---------------------------------------------------------------------------------
***************************************
(1803, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1803,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 50400 candidates, totalling 151200 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
67200 fits failed out of a total of 151200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
67200 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 1320, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 304, in fit
    raise ValueError("min_weight_fraction_leaf must in [0, 0.5]")
ValueError: min_weight_fract

0 Best_hyperparameter_Arvore {'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}
***************************************
(1796, 10) x1_train
*****************++*********************
(731, 10) x1_test
******************---********************
(1796,) y1_train
****************1234*********************
(731,) y1_test
*****************asddfg******************
 -----------------grid_search.fit--------------
Fitting 3 folds for each of 50400 candidates, totalling 151200 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
67200 fits failed out of a total of 151200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
67200 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 1320, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 304, in fit
    raise ValueError("min_weight_fraction_leaf must in [0, 0.5]")
ValueError: min_weight_fract

1 Best_hyperparameter_Arvore {'max_depth': 3, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'splitter': 'random'}
---------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [-0.864 -0.869 -0.864 ...    nan    nan    nan]
  category=UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the train scores are non-finite: [-0.76  -0.802 -0.76  ...    nan    nan    nan]
  category=UserWarning,
